In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#NLP Imports
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv('../project-5/clean_df.csv')
df

,review,sentiment,review_clean
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production the filming tech...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically there s a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei s love in the time of money is a...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,i thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,i am a catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,i m going to have to disagree with the previou...


In [4]:
lemmatizer = WordNetLemmatizer()

In [9]:
tokenizer = RegexpTokenizer('\s+', gaps=True)

In [10]:
df['review_clean_list'] = df['review_clean'].apply(tokenizer.tokenize)
df.head()

,review,sentiment,review_clean,review_clean_list
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,"[one, of, the, other, reviewers, has, mentione..."
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production the filming tech...,"[a, wonderful, little, production, the, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,"[i, thought, this, was, a, wonderful, way, to,..."
3,Basically there's a family where a little boy ...,negative,basically there s a family where a little boy ...,"[basically, there, s, a, family, where, a, lit..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei s love in the time of money is a...,"[petter, mattei, s, love, in, the, time, of, m..."


In [15]:
df['review_lem'] = df['review_clean_list'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
df.head()

,review,sentiment,review_clean,review_clean_list,review_lem
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...,"[one, of, the, other, reviewers, has, mentione...","[one, of, the, other, reviewer, ha, mentioned,..."
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production the filming tech...,"[a, wonderful, little, production, the, filmin...","[a, wonderful, little, production, the, filmin..."
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...,"[i, thought, this, was, a, wonderful, way, to,...","[i, thought, this, wa, a, wonderful, way, to, ..."
3,Basically there's a family where a little boy ...,negative,basically there s a family where a little boy ...,"[basically, there, s, a, family, where, a, lit...","[basically, there, s, a, family, where, a, lit..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei s love in the time of money is a...,"[petter, mattei, s, love, in, the, time, of, m...","[petter, mattei, s, love, in, the, time, of, m..."


In [16]:
pd.set_option('max_colwidth', 800)
df[['review_clean_list', 'review_lem']]

,review_clean_list,review_lem
0,"[one, of, the, other, reviewers, has, mentioned, that, after, watching, just, 1, oz, episode, you, ll, be, hooked, they, are, right, as, this, is, exactly, what, happened, with, me, the, first, thing, that, struck, me, about, oz, was, its, brutality, and, unflinching, scenes, of, violence, which, set, in, right, from, the, word, go, trust, me, this, is, not, a, show, for, the, faint, hearted, or, timid, this, show, pulls, no, punches, with, regards, to, drugs, sex, or, violence, its, is, hardcore, in, the, classic, use, of, the, word, it, is, called, oz, as, that, is, the, nickname, given, to, the, ...]","[one, of, the, other, reviewer, ha, mentioned, that, after, watching, just, 1, oz, episode, you, ll, be, hooked, they, are, right, a, this, is, exactly, what, happened, with, me, the, first, thing, that, struck, me, about, oz, wa, it, brutality, and, unflinching, scene, of, violence, which, set, in, right, from, the, word, go, trust, me, this, is, not, a, show, for, the, faint, hearted, or, timid, this, show, pull, no, punch, with, regard, to, drug, sex, or, violence, it, is, hardcore, in, the, classic, use, of, the, word, it, is, called, oz, a, that, is, the, nickname, given, to, the, ...]"
1,"[a, wonderful, little, production, the, filming, technique, is, very, unassuming, very, old, time, bbc, fashion, and, gives, a, comforting, and, sometimes, discomforting, sense, of, realism, to, the, entire, piece, the, actors, are, extremely, well, chosen, michael, sheen, not, only, has, got, all, the, polari, but, he, has, all, the, voices, down, pat, too, you, can, truly, see, the, seamless, editing, guided, by, the, references, to, williams, diary, entries, not, only, is, it, well, worth, the, watching, but, it, is, a, terrificly, written, and, performed, piece, a, masterful, production, about, one, of, the, great, master, s, of, comedy, and, his, life, ...]","[a, wonderful, little, production, the, filming, technique, is, very, unassuming, very, old, time, bbc, fashion, and, give, a, comforting, and, sometimes, discomforting, sense, of, realism, to, the, entire, piece, the, actor, are, extremely, well, chosen, michael, sheen, not, only, ha, got, all, the, polari, but, he, ha, all, the, voice, down, pat, too, you, can, truly, see, the, seamless, editing, guided, by, the, reference, to, williams, diary, entry, not, only, is, it, well, worth, the, watching, but, it, is, a, terrificly, written, and, performed, piece, a, masterful, production, about, one, of, the, great, master, s, of, comedy, and, his, life, ...]"
2,"[i, thought, this, was, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, sitting, in, the, air, conditioned, theater, and, watching, a, light, hearted, comedy, the, plot, is, simplistic, but, the, dialogue, is, witty, and, the, characters, are, likable, even, the, well, bread, suspected, serial, killer, while, some, may, be, disappointed, when, they, realize, this, is, not, match, point, 2, risk, addiction, i, thought, it, was, proof, that, woody, allen, is, still, fully, in, control, of, the, style, many, of, us, have, grown, to, love, this, was, the, most, i, d, laughed, at, one, of, woody, s, ...]","[i, thought, this, wa, a, wonderful, way, to, spend, time, on, a, too, hot, summer, weekend, sitting, in, the, air, conditioned, theater, and, watching, a, light, hearted, comedy, the, plot, is, simplistic, but, the, dialogue, is, witty, and, the, character, are, likable, even, the, well, bread, suspected, serial, killer, while, some, may, be, disappointed, when, they, realize, this, is, not, match, point, 2, risk, addiction, i, thought, it, wa, proof, that, woody, allen, is, still, fully, in, control, of, the, style, many, of, u, have, grown, to, love, this, wa, the, most, i, d, laughed, at, one, of, woody, s, ...]"
3,"[basically, there, s, a, family, where, a, little, boy, jake, thinks, there, s, a, zombie, in, his, closet, his, parents, are, fighting, all, the, time, 